In [2]:
%matplotlib inline

from pandas import DataFrame, read_sas
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

from SECEdgar.crawler import SecCrawler

In [3]:
# Read in SAS data set - takes a while...
%time data = read_sas("data/crsp_comp.sas7bdat")

Wall time: 3min 19s


In [4]:
# List column names
list(data.columns.values)

[u'CUSIP',
 u'PERMNO',
 u'cik',
 u'date',
 u'year',
 u'fyear',
 u'tic',
 u'conm',
 u'mkvalt',
 u'at',
 u'intan',
 u'lt',
 u'book_market',
 u'tlta',
 u'cacl',
 u'nita',
 u'icf_na',
 u'rsiz',
 u'mkt_val',
 u'PRC',
 u'RET',
 u'ASKHI',
 u'BIDLO',
 u'VOL',
 u'RETX',
 u'vwretd',
 u'totval']

In [5]:
# Sort the set by cusip, permno, cik, and then year (descending)
data.sort_values(['CUSIP', 'PERMNO', 'cik', 'year'], ascending=[True, True, True, False], inplace=True)

In [6]:
# Drop everything except CUSIP, PERMNO, and CIK and remove any duplicates, then drop everything except CIK column
ciks = data.drop(data.columns[3:], axis=1).drop_duplicates().drop(data.columns[0:2], axis=1)

# Now, add in the ticker to the dataframe
ciks['tic'] = data['tic']

In [7]:

# Iterate over each CIK and pull the relevant 10k filings
crawler = SecCrawler()
end_date = '20081231'
count = '20'

for index, row in ciks.iterrows():
    cik = row.iloc[0]
    tic = row.iloc[1]

    crawler.filing_10K(tic, cik, end_date, count)


('Skipping %s (%s)...', '3ANAFE', '0000855569')
('Skipping %s (%s)...', 'AAON', '0000824142')
('Skipping %s (%s)...', 'AIR', '0000001750')
('Skipping %s (%s)...', 'ABB', '0001091587')
('Skipping %s (%s)...', 'ABCRQ', '0000913364')
('Skipping %s (%s)...', 'ABTC', '0000902476')
('Skipping %s (%s)...', 'ACCC.', '0000783233')
('Skipping %s (%s)...', 'ACCO', '0000712034')
('Skipping %s (%s)...', 'ACMR', '0001042809')
('Skipping %s (%s)...', 'BRR', '0000711307')
('Skipping %s (%s)...', 'AYS.', '0001047113')
('Skipping %s (%s)...', 'ADCO.', '0000934649')
('Skipping %s (%s)...', 'ADCT', '0000061478')
('Skipping %s (%s)...', 'IATV', '0000854152')
('Skipping %s (%s)...', 'ADAM.1', '0000863650')
('Skipping %s (%s)...', 'ADEX', '0000884498')
('Skipping %s (%s)...', 'ADT.1', '0000776873')
('Skipping %s (%s)...', 'AERN', '0000863872')
('Skipping %s (%s)...', 'AFCX', '0000913360')
('Skipping %s (%s)...', 'ACTT', '0000918709')
('Skipping %s (%s)...', 'ABM', '0000771497')
('Skipping %s (%s)...', 'AMNUQ

c:\python27\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file c:\python27\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


('Number of files to download %s', 16)
Starting download....
Not able to save the file :( 
Successfully downloaded all the files
Not able to create directory
started !0-KAEOK
('Number of files to download %s', 3)
Starting download....
Not able to save the file :( 
Successfully downloaded all the files
Not able to create directory
started !0-KALXN
('Number of files to download %s', 18)
Starting download....
Not able to save the file :( 
Successfully downloaded all the files
Not able to create directory
started !0-KALXA
('Number of files to download %s', 3)
Starting download....
Not able to save the file :( 
Successfully downloaded all the files
Not able to create directory
started !0-KALFO
('Number of files to download %s', 0)
Starting download....
Successfully downloaded all the files
Not able to create directory
started !0-KALGO
('Number of files to download %s', 6)
Starting download....
Not able to save the file :( 
Successfully downloaded all the files
Not able to create directory
s

KeyboardInterrupt: 